In [1]:
import pandas as pd
import numpy as np
from HFC32_hmimFAP_PR import configuration
from pyomo.environ import (ConcreteModel,
                           SolverFactory,
                           units as pyunits)

import idaes.logger as idaeslog
import pyomo.contrib.parmest.parmest as parmest

import pytest

from idaes.core import FlowsheetBlock
from idaes.generic_models.properties.core.generic.generic_property import (
        GenericParameterBlock)

from idaes.core.util.model_statistics import degrees_of_freedom

In [2]:
def PR_model(data):
    """
    Function which returns initilized model.

    Args:
        data: pandas DataFrame with data

    Returns:
        initialized model
    """
    eps = 0.1
    init_kappa_2_1A = -0.01
    init_kappa_1_2A = -0.02
    m = ConcreteModel()

    m.fs = FlowsheetBlock(default={"dynamic": False})

    m.fs.properties = GenericParameterBlock(default=configuration)

    m.fs.state_block = m.fs.properties.state_block_class(
        default={"parameters": m.fs.properties,
                 "defined_state": True})
    x = float(data['x_R32'])+eps
    m.fs.state_block.flow_mol.fix(1)
    m.fs.state_block.temperature.fix(float(data["temperature"]))
    m.fs.state_block.pressure.fix(float(data["pressure"]))
    m.fs.state_block.mole_frac_comp['hmimFAP'].fix(1-x)
    m.fs.state_block.mole_frac_comp['R32'].fix(x)

    # parameter - kappa_A_ij (set at 0.3, 0 if i=j)
    m.fs.properties.PR_kappa_A['hmimFAP','hmimFAP'].fix(0)
    m.fs.properties.PR_kappa_A['hmimFAP', 'R32'].fix(init_kappa_2_1A)
    m.fs.properties.PR_kappa_A['R32', 'R32'].fix(0)
    m.fs.properties.PR_kappa_A['R32', 'hmimFAP'].fix(init_kappa_1_2A)
    m.fs.properties.PR_kappa_B['hmimFAP', 'hmimFAP'].fix(0)
    m.fs.properties.PR_kappa_B['hmimFAP', 'R32'].fix(0)
    m.fs.properties.PR_kappa_B['R32', 'R32'].fix(0)
    m.fs.properties.PR_kappa_B['R32', 'hmimFAP'].fix(0)
    m.fs.properties.PR_kappa_C['hmimFAP', 'hmimFAP'].fix(0)
    m.fs.properties.PR_kappa_C['hmimFAP', 'R32'].fix(0)
    m.fs.properties.PR_kappa_C['R32', 'R32'].fix(0)
    m.fs.properties.PR_kappa_C['R32', 'hmimFAP'].fix(0)
    # Initialize the flash unit
    m.fs.state_block.initialize(outlvl=idaeslog.CRITICAL)

    # Fix the state variables on the state block
    m.fs.state_block.pressure.unfix()
    m.fs.state_block.mole_frac_comp['hmimFAP'].unfix()
    m.fs.state_block.mole_frac_comp['R32'].unfix()
    m.fs.state_block.temperature.fix(float(data["temperature"]))
    m.fs.state_block.mole_frac_phase_comp['Liq', 'R32'].fix(float(data['x_R32']))
    m.fs.state_block.mole_frac_phase_comp['Liq', 'hmimFAP'].fix(float(data['x_hmimFAP']))
    m.fs.state_block.mole_frac_comp['R32'].fix(float(data['x_R32'])+eps)
    m.fs.state_block.mole_frac_comp['hmimFAP'].unfix()

    # Set bounds on variables to be estimated
    m.fs.properties.PR_kappa_A['hmimFAP', 'R32'].setlb(-5)
    m.fs.properties.PR_kappa_A['hmimFAP', 'R32'].setub(5)

    m.fs.properties.PR_kappa_A['R32', 'hmimFAP'].setlb(-5)
    m.fs.properties.PR_kappa_A['R32', 'hmimFAP'].setub(5)

    # Return initialized flash model
    return m

def SSE(m, data):
    """
    returns objective function expresion.

    Args:
        m: model
        data: pandas DataFrame with data

    Returns:
        objectuve function scaled expresion
    """
    expr = ((float(data["pressure"]) - m.fs.state_block.pressure)**2)

    return expr * 1e-6

variable_name = ["fs.properties.PR_kappa_A['hmimFAP', 'R32']",
                 "fs.properties.PR_kappa_A['R32', 'hmimFAP']"]


In [3]:
data = pd.read_csv('R32_hmimFAP.csv')

print(data)

   temperature  pressure  x_R32  x_hmimFAP
0       298.15     50000  0.067      0.933
1       298.15    100000  0.126      0.874
2       298.15    200000  0.233      0.767
3       298.15    400000  0.402      0.598
4       298.15    600000  0.531      0.469
5       298.15    800000  0.634      0.366
6       298.15   1000000  0.720      0.280


In [4]:
N = 100 #number of samples
results_params = np.zeros((N,2))
results_obj = np.zeros((N,1))

st_dev_T = 0.01 #C
st_dev_P = 9771 #MPa
st_dev_x = 0.10 #mole fraction

for i in range(N):
    print(i)
    noise_x = np.random.normal(loc=0,scale=st_dev_x)
    print(noise_x)
    data_ = data.copy()
    data_["x_hmimFAP"] += data_["x_hmimFAP"] * noise_x
    data_.loc[data_['x_hmimFAP'] <= 0.0001, 'x_hmimFAP'] = 0.0001
    data_.loc[data_['x_hmimFAP'] >= 0.9999 , 'x_hmimFAP'] = 0.9999      

    try:
        pest = parmest.Estimator(PR_model, data_, variable_name, SSE, tee=False)

        obj_value, parameters = pest.theta_est()

        for k,v in parameters.items():
            print(k, "=", v)

        line = 0

        for k,v in parameters.items():
            results_params[i,line] = v
            line += 1
        results_obj[i] = obj_value

    except ValueError:
        results_params[i,0] = 'NaN'
        results_params[i,1] = 'NaN'


0
0.049413716013364105
2022-03-06 12:14:14 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 12:14:15 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 12:14:16 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 12:14:17 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 12:14:18 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 12:14:19 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 12:14:20 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
    model.name="_Q_opt";
      - termination condition: infeasible
      - message from solver: Ipopt 3.13.2\x3a Converged to a lo

fs.properties.PR_kappa_A[hmimFAP,R32] = 0.491245102520228
fs.properties.PR_kappa_A[R32,hmimFAP] = 0.021202462799302955
7
0.03758973973076929
2022-03-06 12:15:27 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 12:15:28 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 12:15:30 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 12:15:31 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 12:15:32 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 12:15:33 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 12:15:34 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
    model.na

2022-03-06 12:16:41 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 12:16:42 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 12:16:43 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 12:16:44 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
fs.properties.PR_kappa_A[hmimFAP,R32] = -0.01581633814537737
fs.properties.PR_kappa_A[R32,hmimFAP] = -0.025462598740652445
15
0.022456588535137774
2022-03-06 12:16:47 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 12:16:48 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 12:16:49 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-0

2022-03-06 12:17:52 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 12:17:53 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
    model.name="_Q_opt";
      - termination condition: infeasible
      - message from solver: Ipopt 3.13.2\x3a Converged to a locally
        infeasible point. Problem may be infeasible.
fs.properties.PR_kappa_A[hmimFAP,R32] = -0.0038472013951813583
fs.properties.PR_kappa_A[R32,hmimFAP] = -0.023539572195588734
22
0.03231000806957487
2022-03-06 12:17:57 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 12:17:58 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 12:17:59 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 12:18:00 [INFO] idaes.init.fs.state_block: Prop

2022-03-06 12:19:02 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 12:19:03 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
    model.name="_Q_opt";
      - termination condition: infeasible
      - message from solver: Ipopt 3.13.2\x3a Converged to a locally
        infeasible point. Problem may be infeasible.
fs.properties.PR_kappa_A[hmimFAP,R32] = -0.015075219565104098
fs.properties.PR_kappa_A[R32,hmimFAP] = -0.025461571417293216
29
0.012885533781538616
2022-03-06 12:19:07 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 12:19:08 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 12:19:09 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 12:19:10 [INFO] idaes.init.fs.state_block: Prop

...

2022-03-06 12:25:14 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 12:25:15 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
    model.name="_Q_opt";
      - termination condition: infeasible
      - message from solver: Ipopt 3.13.2\x3a Converged to a locally
        infeasible point. Problem may be infeasible.
fs.properties.PR_kappa_A[hmimFAP,R32] = 0.48223073422551965
fs.properties.PR_kappa_A[R32,hmimFAP] = 0.024331863173280444
66
-0.06711277371374169
2022-03-06 12:25:19 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 12:25:21 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 12:25:22 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 12:25:23 [INFO] idaes.init.fs.state_block: Propert

fs.properties.PR_kappa_A[hmimFAP,R32] = 0.5740895901155729
fs.properties.PR_kappa_A[R32,hmimFAP] = 0.00796299021739378
73
-0.05188432245696985
2022-03-06 12:26:35 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 12:26:36 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 12:26:38 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 12:26:40 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 12:26:42 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 12:26:43 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 12:26:44 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
fs.propert

fs.properties.PR_kappa_A[hmimFAP,R32] = -0.01721086770007712
fs.properties.PR_kappa_A[R32,hmimFAP] = -0.02546709490829017
81
0.11265398779396119
2022-03-06 12:28:11 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 12:28:12 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 12:28:14 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 12:28:15 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 12:28:17 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 12:28:19 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 12:28:20 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
    mode

2022-03-06 12:29:57 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 12:29:58 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 12:29:59 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 12:30:01 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
fs.properties.PR_kappa_A[hmimFAP,R32] = -0.015891160268562535
fs.properties.PR_kappa_A[R32,hmimFAP] = -0.025462813498266394
89
-0.08612615504247055
2022-03-06 12:30:04 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 12:30:06 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 12:30:07 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-

2022-03-06 12:31:22 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 12:31:22 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 12:31:23 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 12:31:25 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 12:31:26 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 12:31:27 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
    model.name="_Q_opt";
      - termination condition: infeasible
      - message from solver: Ipopt 3.13.2\x3a Converged to a locally
        infeasible point. Problem may be infeasible.
fs.properties.PR_kappa_A[hmimFAP,R32] = -0.01455218374747333
fs.properties.PR_kappa_A

In [5]:
print(results_params)

[[-0.01472437 -0.02546231]
 [-0.006689   -0.02404128]
 [-0.01697585 -0.02546626]
 [ 0.48110435  0.02397131]
 [-0.01408131 -0.02546359]
 [ 0.031704   -0.01752782]
 [ 0.4912451   0.02120246]
 [-0.01486799 -0.02546201]
 [-0.01583727 -0.02546266]
 [-0.01480812 -0.02546214]
 [-0.01579816 -0.02546255]
 [-0.0163731  -0.02546426]
 [-0.01431315 -0.02546314]
 [-0.01616243 -0.02546362]
 [-0.01581634 -0.0254626 ]
 [-0.01505205 -0.02546162]
 [-0.01443535 -0.02546289]
 [-0.01516384 -0.02546138]
 [-0.01623803 -0.02546385]
 [ 0.50256852  0.01980983]
 [-0.01992802 -0.02548015]
 [-0.0038472  -0.02353957]
 [-0.00715901 -0.02414905]
 [-0.01650895 -0.02546469]
 [-0.01529504 -0.02546111]
 [-0.01124193 -0.02541993]
 [-0.01422655 -0.02546331]
 [-0.01680202 -0.02546566]
 [-0.01507522 -0.02546157]
 [-0.01516861 -0.02546137]
 [-0.01482769 -0.0254621 ]
 [-0.01423416 -0.02546329]
 [-0.01540623 -0.02546147]
 [-0.01536785 -0.02546137]
 [-0.01441192 -0.02546294]
 [ 0.47777682  0.02289311]
 [-0.01614511 -0.02546356]
 

In [6]:
results = pd.DataFrame(results_params)
results.to_csv('R32_hmimFAP_10_2.csv')